# OpenAIのチャットAPIの基礎

# 準備

In [ ]:
!pip install openai==1.40.6
!pip install httpx==0.27.2

In [ ]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

## 2.5 Chat Completions APIのハンズオン


In [ ]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "こんにちは!私はジョンといいます！"},
        {"role": "assistant", "content": "こんにちは、ジョンさん！どのようにお手伝いできますか？"},
        {"role": "user", "content": "私の名前はわかりますか？"},
    ],
)

print(response.to_json(indent=2))


{
  "id": "chatcmpl-ApyDbqWxuQLJqNrar8vTHO0yb3WVy",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "はい、あなたの名前はジョンさんです。何か特別なことについてお話ししたいことがありますか？",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1736949571,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_72ed7ab54c",
  "usage": {
    "completion_tokens": 28,
    "prompt_tokens": 60,
    "total_tokens": 88,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  }
}


### ストリーミング

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "こんにちは!私はジョンといいます！"},
    ],
    stream=True
)

for chunk in response:
    content = chunk.choices[0].delta.content
    if content is not None:
        print(content, end="", flush=True)

こんにちは、ジョンさん！お話しできてうれしいです。今日はどのようなことをお話ししましょうか？

### JSON モード

In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": '人物一覧を次のJSON形式で出力してください。\n{"people": ["aaa", "bbb"]}',
        },
        {
            "role": "user",
            "content":"昔あるところにおじいさんとおばあさんがいました",
        },
    ],
    response_format={"type": "json_object"},
)
print(response.choices[0].message.content)

<class 'str'>


### Vision (画像入力)

In [ ]:
from openai import OpenAI

client = OpenAI()

image_url = "https://raw.githubusercontent.com/yoshidashingo/langchain-book/main/assets/cover.jpg"

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type":"text", "text": "画像を説明してください。"},
                {"type":"image_url", "image_url": {"url": image_url}},
            ],
        }
    ],
)

print(response.choices[0].message.content)

この画像は書籍の表紙を示しており、タイトルは「ChatGPT/LangChainによるチャットシステム構築【実践】入門」となっています。著者名は「吉田真吾、大櫃勇樹」と記されています。表紙にはカラフルな鳥のイラストと「ChatGPT」というテキストが目立つ大きなフォントで表示されています。また、書籍の内容については、大規模言語モデルを活用するための基礎知識と実践について説明しています。


## 2.6 Function calling

In [ ]:
import json

def get_current_weather(location, unit="fahrenheit"):
    if "tokyo" in location.lower():
      return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
      return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
      return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
      return json.dumps({"location": location, "temperature": "unknown"})

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

from openai import OpenAI

client = OpenAI()

messages = [
    {"role": "user", "content": "東京の天気はどうですか？"},
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
)

response_message = response.choices[0].message
messages.append(response_message.to_dict())

available_functions = {
    "get_current_weather": get_current_weather,
}

for tool_call in response_message.tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    print(function_args)
    function_response = function_to_call(
        location=function_args.get("location"),
        unit=function_args.get("unit"),
    )
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
    )

#print(messages)

second_response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)

#print(second_response.to_json(indent=2))


{'location': '東京, 日本', 'unit': 'celsius'}
